In [1]:
## ADVENT OF CODE 2022, Day 7
## Edmund Dickinson, Python implementation

# File read
input_folder = "Input/"
input_file = "day7a.txt"
file_path = input_folder + input_file

with open(file_path) as file:
    input = file.read().splitlines()

In [2]:
# File processing
# Generate equal length lists of instructions and responses, as strings
output_str = []
instructions = []
responses = []
output_expected = False

for line in input:
    if (line[0] == '$'):
        # Console input (instruction)
        # Write output buffer from previous instruction (if any) and clear
        if(output_expected):
            responses.append(output_str)
            output_str = []
        
        # Discard $ prompt
        instructions.append(line[2:])
        output_expected = True
    else:
        # Console output
        # Add present line to output buffer
        output_str.append(line)
# Add last output if present
if(output_expected):
    responses.append(output_str)

In [3]:
# File system class structure
class FileSystem:
    def __init__(self, name, size=0, parent=None):        
        self.name = name
        self.parent = parent                   

class Dir(FileSystem):
    def __init__(self, name, size=0, parent=None):
        # Directories have no size but can have children
        super().__init__(name, parent=parent)       
        self.children = []

    def addChild(self, name, type, size=0):
        # Should check for name clash with existing children
        exists = False
        for child in self.children:
            if (name == child.name ):
                exists = True

        if (exists == False):
            # Doesn't already exists, so add it
            if(type == "dir"):
                self.children.append(Dir(name, parent=self))
            elif(type == "file"):
                self.children.append(File(name, size, parent=self))
            else:
                raise Exception("Invalid type.")
        else:
            # Already exists, handle if necessary
            None

    def getChild(self, name):
        match = [x for x in self.children if x.name == name]
        if (len(match) > 1):
            raise Exception("Multiple matches")
        elif (len(match) == 0):
            # No matches
            return None
        else:
            return match[0]
    
    def getSize(self):
        size_tot = 0
        for child in self.children:
            size_tot = size_tot + child.getSize()
        return size_tot

    def getSubDir(self):
        # Returns a list of directory and all of its subdirectories
        dirs = [ self ]
        for child in self.children:
            dirs.extend(child.getSubDir())
        return dirs

class File(FileSystem):
    # Files have size but no children
    def __init__(self, name, size, parent=None):
        # File creation must be called with size
        super().__init__(name, size, parent=parent)
        self.size = size

    def getSize(self):
        # Files have no children 
        return self.size

    def getSubDir(self):
        # Files are not directories
        return []

In [4]:
# Analysis
root = Dir("root")
currdir = root

for k, item in enumerate(instructions):
    response = responses[k]
    
    if item[:2] == "ls":
        for line in response:
            type, name = line.split()
            if (type == "dir"):
                currdir.addChild(name, "dir")
            else:
                # The type field is the size of the file in bytes
                size = int(type);
                currdir.addChild(name, "file", size)
    elif item[:2] == "cd":
        name = item.split()[1]
        if (name == ".."):
            # Go to parent directory
            currdir = currdir.parent
        elif (name == "/"):
            # Go back to root directory
            currdir = root
        else:
            # Go down to stated directory
            currdir = currdir.getChild(name)

# Get all directories and subdirectories
dirs = root.getSubDir()

# Max size limit (Puzzle One)
sizelim = 100000
# Note(ED): would love to know how to print the names of these in a single pass
dirsize_search = [i.getSize() for i in dirs if i.getSize() <= sizelim]
print("Sum of directories with size at most", sizelim, ":", sum(dirsize_search))

# Min size limit (Puzzle Two)
sys_size = 70_000_000
need_size = 30_000_000
unused_size = sys_size - root.getSize()
del_size = need_size - unused_size
dirsize_del = [i.getSize() for i in dirs if i.getSize() >= del_size]
print("Smallest directory with size at least", del_size, ":", min(dirsize_del))

Sum of directories with size at most 100000 : 1749646
Smallest directory with size at least 1412830 : 1498966
